In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`, load it here
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) Upload back up with `make csv_to_s3`
# (5) Run `heroku run make importdb` to get it to production
links = pd.read_csv('data/export.csv')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77588,134133,https://www.slowboring.com/p/saturday-thread-643,Saturday Thread,\N,\N,2021-10-09 16:30:29.010475+00,2021-10-09 16:30:29.010475+00,NaN,\N,Yglesias,60.0,0.0
77589,134134,https://thedispatch.com/p/all-the-time-in-the-...,All the Time in the World,\N,\N,2021-10-09 16:30:29.015698+00,2021-10-09 16:30:29.015698+00,NaN,\N,Dispatch,16.0,0.0
77590,134135,https://statmodeling.stat.columbia.edu/2021/10...,Fun example of an observational study: Effect...,\N,\N,2021-10-09 16:30:29.01827+00,2021-10-09 16:30:29.01827+00,NaN,\N,Gelman,98.0,0.0
77591,134136,https://www.alignmentforum.org/posts/YbAZ4nSA8...,"[AN #166]: Is it crazy to claim we""re in the m...",\N,\N,2021-10-09 16:30:29.020367+00,2021-10-09 16:30:29.020367+00,NaN,\N,AlignmentF,99.0,0.0


In [2]:
links['added'].max()

'2021-10-10 04:30:13+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
77466    134010
77481    134026
77482    134027
77483    134028
77592    134138
Name: id, Length: 3685, dtype: int64

In [4]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [5]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 100 == 0:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/54


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77642,134188,https://www.macfound.org/learning/,What We're Learning - MacArthur Foundation - M...,\N,\N,2021-10-10 00:32:11.994423,2021-10-10 00:32:11.994436,\N,\N,Custom,66.0,0.0
77643,134189,https://twitter.com/TimHarford/status/14460033...,https://twitter.com/TimHarford/status/14460033...,\N,\N,2021-10-10 00:32:12.003566,2021-10-10 00:32:12.003578,\N,\N,Custom,21.0,0.0
77644,134190,https://twitter.com/TrungTPhan/status/14457680...,https://twitter.com/TrungTPhan/status/14457680...,\N,\N,2021-10-10 00:32:12.013214,2021-10-10 00:32:12.013226,\N,\N,Custom,92.0,0.0
77645,134191,https://www.pnas.org/content/118/41/e2021636118,Slowed canonical progress in large fields of s...,\N,\N,2021-10-10 00:32:12.023233,2021-10-10 00:32:12.023246,\N,\N,Custom,72.0,0.0


In [6]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77642,134188,https://www.macfound.org/learning/,What We're Learning - MacArthur Foundation - M...,\N,\N,2021-10-10 00:32:11.994423,2021-10-10 00:32:11.994436,\N,\N,Custom,66.0,0.0
77643,134189,https://twitter.com/TimHarford/status/14460033...,https://twitter.com/TimHarford/status/14460033...,\N,\N,2021-10-10 00:32:12.003566,2021-10-10 00:32:12.003578,\N,\N,Custom,21.0,0.0
77644,134190,https://twitter.com/TrungTPhan/status/14457680...,https://twitter.com/TrungTPhan/status/14457680...,\N,\N,2021-10-10 00:32:12.013214,2021-10-10 00:32:12.013226,\N,\N,Custom,92.0,0.0
77645,134191,https://www.pnas.org/content/118/41/e2021636118,Slowed canonical progress in large fields of s...,\N,\N,2021-10-10 00:32:12.023233,2021-10-10 00:32:12.023246,\N,\N,Custom,72.0,0.0


In [7]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
77642,134188,https://www.macfound.org/learning/,What We're Learning - MacArthur Foundation - M...,\N,\N,2021-10-10 00:32:11.994423,2021-10-10 00:32:11.994436,\N,\N,Custom,66,0
77643,134189,https://twitter.com/TimHarford/status/14460033...,https://twitter.com/TimHarford/status/14460033...,\N,\N,2021-10-10 00:32:12.003566,2021-10-10 00:32:12.003578,\N,\N,Custom,21,0
77644,134190,https://twitter.com/TrungTPhan/status/14457680...,https://twitter.com/TrungTPhan/status/14457680...,\N,\N,2021-10-10 00:32:12.013214,2021-10-10 00:32:12.013226,\N,\N,Custom,92,0
77645,134191,https://www.pnas.org/content/118/41/e2021636118,Slowed canonical progress in large fields of s...,\N,\N,2021-10-10 00:32:12.023233,2021-10-10 00:32:12.023246,\N,\N,Custom,72,0


In [8]:
links.to_csv('data/export.csv', index=False)